# CONCAT _0, _1 zarr to have only on file per type

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram
import warnings
warnings.filterwarnings("ignore")

import os
from glob import glob

import histlib.box as box
import histlib.aviso as aviso
import histlib.cstes as cstes
from histlib.cstes import labels, zarr_dir

In [3]:
zarr_dir_ok = zarr_dir+'_ok'

- i=0,1 :  jobs=6, cores = 26 
- i=2,3 :  jobs=1, cores = 26 
- i=4 :    jobs=1, cores = 10
- i=5 :    jobs=1, cores = 26
- i=6 :    jobs=4, cores = 26

In [4]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=10, processes=10, walltime='04:00:00')
    w = cluster.scale(jobs=3)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.70:8787/status,
Dashboard: http://10.148.1.70:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.70:40301,Workers: 0
Dashboard: http://10.148.1.70:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# AVISO

In [5]:
for l in labels:
    files = sorted(glob(os.path.join(zarr_dir, f'aviso_{l}_*.zarr')))
    for f in files :
        ds = xr.open_dataset(f).chunk({'obs':500})
        if ds.dims['aviso_time'] ==25 :#keep only five days of aviso (error for first drifter type, corrected after because too long)
            ds = ds.isel(aviso_time=slice(10, 15))
        if '_0.' in f :#overwrite
            ds.to_zarr(os.path.join(zarr_dir_ok, 'aviso', f'aviso_{l}.zarr'), mode = 'w')
        else : #append_dim
            ds.to_zarr(os.path.join(zarr_dir_ok,'aviso', f'aviso_{l}.zarr'), append_dim='obs')
    print(l)

# ERASTAR

In [ ]:
for l in labels:
    files = sorted(glob(os.path.join(zarr_dir, f'erastar_{l}_*.zarr')))
    for f in files :
        ds = xr.open_dataset(f).chunk({'obs':500})
        if '_0.' in f :#overwrite
            ds.to_zarr(os.path.join(zarr_dir_ok, 'erastar', f'erastar_{l}.zarr'), mode = 'w')
        else : #append_dim
            ds.to_zarr(os.path.join(zarr_dir_ok,'erastar', f'erastar_{l}.zarr'), append_dim='obs')
    print(l)

# TEST

In [15]:
ds = xr.open_dataset(os.path.join(zarr_dir_ok,f'aviso_{l}.zarr')).chunk({'obs':500}).persist()

In [6]:
cluster.close()

In [ ]:
ds=xr